![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Rollout チュートリアル <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/rollout.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

このノートブックは、ネイティブPythonバインディングを使用した [**MuJoCo** 物理エンジン](https://github.com/google-deepmind/mujoco#readme) のチュートリアルです。

このノートブックでは、MuJoCo Pythonライブラリに含まれる `rollout` モジュールについて説明します。このモジュールは、C++関数を内部で使用してシミュレーションの「ロールアウト」を実行します。ロールアウトはマルチスレッドで実行できます。

以下では、まず各引数の使い方を例を交えて説明します。次に、高度なユースケースの例をいくつか紹介します。最後に、 `rollout` を純粋なPythonおよびMJXとベンチマーク比較します。

なお、ベンチマークは16スレッド以上のCPUとRTX 4090またはA100で実行するように設計されています。一般的な無料のColabランタイムでは、妥当な時間内に完了しません。

<!-- Copyright 2025 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

# すべてのインポート

In [0]:
!pip install mujoco
!pip install mujoco_mjx
!pip install brax

# GPUレンダリングのセットアップ。
#from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# glvndがNvidia EGLドライバーを検出できるようにICD設定を追加。
# 通常はNvidiaドライバーパッケージの一部としてインストールされるが、Colab
# カーネルはAPT経由でドライバーをインストールしないため、ICDが欠落している。
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# MuJoCoをEGLレンダリングバックエンドを使用するように設定（GPU必要）
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# インストールが成功したか確認。
try:
  print('Checking that the installation succeeded:')
  import mujoco
  from mujoco import rollout
  from mujoco import mjx
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# XLAにTriton GEMMを使用させる。一部のGPUでsteps/secが約30%向上する
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags

# その他のインポートとヘルパー関数
import copy
import time
from multiprocessing import cpu_count
import threading
import numpy as np
import jax
import jax.numpy as jp

# グラフィックスとプロット。
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib
import matplotlib.pyplot as plt

# numpyからの出力を読みやすくする。
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# スレッド数をmultiprocessingモジュールが報告するCPU数に設定
nthread = cpu_count()

# MuJoCoの標準humanoidとhumanoid_100モデルを取得。
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
humanoid_path = 'mujoco/model/humanoid/humanoid.xml'
humanoid100_path = 'mujoco/model/humanoid/humanoid100.xml'
print('Getting hopper XML description from GitHub:')
!git clone https://github.com/google-deepmind/dm_control
hopper_path ='dm_control/dm_control/suite/hopper.xml'

# インストール時の出力をクリア
from IPython.display import clear_output
clear_output()

# ヘルパー関数

In [0]:
def get_state(model, data, nbatch=1):
  full_physics = mujoco.mjtState.mjSTATE_FULLPHYSICS
  state = np.zeros((mujoco.mj_stateSize(model, full_physics),))
  mujoco.mj_getState(model, data, state, full_physics)
  return np.tile(state, (nbatch, 1))

def xy_grid(nbatch, ncols=10, spacing=0.05):
  nrows = nbatch // ncols
  assert nbatch == nrows * ncols
  xmax = (nrows-1)*spacing/2
  rows = np.linspace(-xmax, xmax, nrows)
  ymax = (ncols-1)*spacing/2
  cols = np.linspace(-ymax, ymax, ncols)
  x, y = np.meshgrid(rows, cols)
  return np.stack((x.flatten(), y.flatten())).T

def benchmark(f, x_list=[None], ntiming=1, f_init=None):
  x_times_list = []
  for x in x_list:
    times = []
    for i in range(ntiming):
      if f_init is not None:
        x_init = f_init(x)

      start = time.perf_counter()
      if f_init is not None:
        f(x, x_init)
      else:
        f(x)
      end = time.perf_counter()
      times.append(end - start)

    x_times_list.append(np.mean(times))
  return np.array(x_times_list)

def render_many(model, data, state, framerate, camera=-1, shape=(480, 640),
                transparent=False, light_pos=None):
  nbatch = state.shape[0]

  if not isinstance(model, mujoco.MjModel):
    model = list(model)

  if isinstance(model, list) and len(model) == 1:
    model = model * nbatch
  elif isinstance(model, list):
    assert len(model) == nbatch
  else:
    model = [model] * nbatch

  # 表示オプション
  vopt = mujoco.MjvOption()
  vopt.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = transparent
  pert = mujoco.MjvPerturb()  # Empty MjvPerturb object
  catmask = mujoco.mjtCatBit.mjCAT_DYNAMIC

  # シミュレーションとレンダリング。
  frames = []
  with mujoco.Renderer(model[0], *shape) as renderer:
    for i in range(state.shape[1]):
      if len(frames) < i * model[0].opt.timestep * framerate:
        for j in range(state.shape[0]):
          mujoco.mj_setState(model[j], data, state[j, i, :],
                             mujoco.mjtState.mjSTATE_FULLPHYSICS)
          mujoco.mj_forward(model[j], data)

          # 最初のモデルでシーンを作成し、後続のモデルを追加
          if j == 0:
            renderer.update_scene(data, camera, scene_option=vopt)
          else:
            mujoco.mjv_addGeoms(model[j], data, vopt, pert, catmask, renderer.scene)

        # リクエストされた場合、ライトを追加
        if light_pos is not None:
          light = renderer.scene.lights[renderer.scene.nlight]
          light.ambient = [0, 0, 0]
          light.attenuation = [1, 0, 0]
          light.castshadow = 1
          light.cutoff = 45
          light.diffuse = [0.8, 0.8, 0.8]
          light.dir = [0, 0, -1]
          light.type = mujoco.mjtLightType.mjLIGHT_SPOT
          light.exponent = 10
          light.headlight = 0
          light.specular = [0.3, 0.3, 0.3]
          light.pos = light_pos
          renderer.scene.nlight += 1

        # レンダリングしてフレームを追加。
        pixels = renderer.render()
        frames.append(pixels)
  return frames

# `rollout` の使い方

`mujoco` Pythonライブラリの `rollout.rollout` 関数は、固定ステップ数のシミュレーションをバッチで実行します。シングルスレッドまたはマルチスレッドモードで実行できます。 `rollout` のユーザーは軽量なスレッドプールの使用を簡単に有効にできるため、純粋なPythonに比べて大幅な高速化が得られます。

以下では、後続の使用例とベンチマークで使用する "tippe top"、"humanoid"、"humanoid100" モデルを読み込みます。

tippe top は [チュートリアルノートブック](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb) からコピーしたものです。humanoid と humanoid100 モデルはMuJoCoに付属しています。

In [0]:
#@title ベンチマークモデル
tippe_top = """
<mujoco model="tippe top">
  <option integrator="RK4"/>

  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="40 40" reflectance=".2"/>
  </asset>

  <worldbody>
    <geom size="1 1 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <camera name="distant" pos="0 -.4 .4" xyaxes="1 0 0 0 1 1"/>
    <body name="top" pos="0 0 .02">
      <freejoint name="top"/>
      <site name="top" pos="0 0 0"/>
      <geom name="ball" type="sphere" size=".02" />
      <geom name="stem" type="cylinder" pos="0 0 .02" size="0.004 .008"/>
      <geom name="ballast" type="box" size=".023 .023 0.005"  pos="0 0 -.015"
       contype="0" conaffinity="0" group="3"/>
    </body>
  </worldbody>

  <sensor>
    <gyro name="gyro" site="top"/>
  </sensor>

  <keyframe>
    <key name="spinning" qpos="0 0 0.02 1 0 0 0" qvel="0 0 0 0 1 200" />
  </keyframe>
</mujoco>
"""

# コマモデルの作成と初期化
top_model = mujoco.MjModel.from_xml_string(tippe_top)
top_data = mujoco.MjData(top_model)
# 状態を回転するコマに設定（キーフレーム0）
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
top_state = get_state(top_model, top_data)

# humanoidモデルの作成と初期化
humanoid_model = mujoco.MjModel.from_xml_path(humanoid_path)
humanoid_data = mujoco.MjData(humanoid_model)
humanoid_data.qvel[2] = 4 # humanoidをジャンプさせる
humanoid_state = get_state(humanoid_model, humanoid_data)

# humanoid100モデルの作成と初期化
humanoid100_model = mujoco.MjModel.from_xml_path(humanoid100_path)
humanoid100_data = mujoco.MjData(humanoid100_model)
h100_state = get_state(humanoid100_model, humanoid100_data)

start = time.time()
top_nstep = int(6 / top_model.opt.timestep)
top_state, _ = rollout.rollout(top_model, top_data, top_state, nstep=top_nstep)

humanoid_nstep = int(3 / humanoid_model.opt.timestep)
humanoid_state, _ = rollout.rollout(humanoid_model, humanoid_data,
                                    humanoid_state, nstep=humanoid_nstep)

humanoid100_nstep = int(3 / humanoid100_model.opt.timestep)
h100_state, _ = rollout.rollout(humanoid100_model, humanoid100_data,
                                       h100_state, nstep=humanoid100_nstep)
end = time.time()

start_render = time.time()
top_frames = render_many(top_model, top_data, top_state, framerate=60, shape=(240, 320))
humanoid_frames = render_many(humanoid_model, humanoid_data, humanoid_state, framerate=120, shape=(240, 320))
humanoid100_frames = render_many(humanoid100_model, humanoid100_data, h100_state, framerate=120, shape=(240, 320))

# humanoidとhumanoid100は半速で表示
media.show_video(np.concatenate((top_frames, humanoid_frames, humanoid100_frames), axis=2), fps=60)
end_render = time.time()

print(f'Rollout took {end-start:.1f} seconds')
print(f'Rendering took {end_render-start_render:.1f} seconds')

## 使い方

始める前に `rollout` のdocstringを読むと役立ちます。要点は、 `rollout` が `nbatch` 個のロールアウトを `nstep` ステップ実行するということです。各 `MjModel` は異なることができますが、パラメータ値の違いのみに限られます。複数の `MjData` を渡すとマルチスレッドが有効になり、 `MjData` 1つにつき1スレッドが使用されます。
詳細なドキュメントは [こちら](https://mujoco.readthedocs.io/en/latest/python.html#rollout) にあります。

次に、最も一般的な引数の使用例を示します。より高度な引数については「高度な使い方」セクションで説明します。

In [0]:
print(rollout.rollout.__doc__)

### 例：異なる初期状態
`rollout` は `nbatch` 個のロールアウトを `nstep` ステップ並列で実行するように設計されています。100個のtippe topを異なる初期回転速度でシミュレーションしてみましょう。

**注意：** rolloutでのマルチスレッドは、以下のようにスレッドごとに1つの MjData を渡すことで有効になります。

In [0]:
nbatch = 100 # シミュレーションするコマの数

# nbatch個の初期状態を取得し、バッチインデックスを使ってtippe topの初期速度をスケーリング
top_data = mujoco.MjData(top_model)
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
initial_states = get_state(top_model, top_data, nbatch)
initial_states[:, -1] *= np.linspace(0.5, 1.5, num=nbatch)

# ロールアウトを実行
start = time.time()
top_datas = [copy.copy(top_data) for _ in range(nthread)] # スレッドごとに1つのMjData
state, sensordata = rollout.rollout(top_model, top_datas, initial_states,
                                    nstep=int(top_nstep*1.5))
end = time.time()

# stateを使ってすべてのコマを一度にレンダリング
start_render = time.time()
framerate = 60
frames = render_many(top_model, top_data, state, framerate, transparent=True)
media.show_video(frames, fps=framerate)
end_render = time.time()

print(f'Rollout time {end-start:.1f} seconds')
print(f'Rendering time {end_render-start_render:.1f} seconds')

このモデルにはコマの中央に角速度センサーがあります。rolloutが返す `sensordata` 配列を使って応答をプロットしてみましょう。

In [0]:
plt.figure(figsize=(12, 8))
plt.subplot(3,1,1)
for i in range(nbatch): plt.plot(sensordata[i, :, 0])
plt.subplot(3,1,2)
for i in range(nbatch): plt.plot(sensordata[i, :, 1])
plt.subplot(3,1,3)
for i in range(nbatch): plt.plot(sensordata[i, :, 2])
plt.show()

### 例：異なるモデル
100個の灰色のコマは少し退屈です。カラフルでサイズも異なる方が良いでしょう！

`rollout` は、次元が同じである限り（つまり、浮動小数点パラメータは異なっていてもよい）、各ロールアウトで異なるモデルを使用できます。同じ初期条件で、サイズと色が異なる100個のtippe topをシミュレーションしてみましょう。

**注意：** 厳密に言えば、モデルは同じ数の状態、制御入力、自由度、センサー出力を持つ必要があります。最も一般的なユースケースは、同じものの複数のモデルで、パラメータ値が異なるケースです。

In [0]:
# 異なる色とサイズの100個のtippe topを作成
nbatch = 100
spec = mujoco.MjSpec.from_string(tippe_top)
spec.lights[0].pos[2] = 2
models = []
for i in range(nbatch):
  for geom in spec.geoms:
    if geom.name in ['ball', 'stem', 'ballast']:
      geom.rgba[:3] = np.random.rand(3)
    if geom.name == 'stem':
      stem_geom = geom
    if geom.name == 'ball':
      ball_geom = geom

  # 元のジオムサイズを保存
  stem_geom_size = np.copy(stem_geom.size)
  ball_geom_size = np.copy(ball_geom.size)

  # ジオムをスケーリングしてモデルをコンパイル
  size_scale = 0.4*np.random.rand(1) + 0.75
  stem_geom.size *= size_scale
  ball_geom.size *= size_scale
  models.append(spec.compile())

  # 元のジオムサイズを復元
  stem_geom.size = stem_geom_size
  ball_geom.size = ball_geom_size

# すべてのコマの初期状態を設定し、グリッド上に配置
top_data = mujoco.MjData(top_model)
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
initial_states = get_state(top_model, top_data, nbatch)
# インデックス0は時間なので、xとyのqpos値はインデックス1と2
initial_states[:, 1:3] = xy_grid(nbatch, ncols=10, spacing=.05)


# ロールアウトを実行
start = time.time()
top_datas = [copy.copy(top_data) for _ in range(nthread)]
nstep = int(9 / top_model.opt.timestep)
state, sensordata = rollout.rollout(models, top_datas, initial_states,
                                    nstep=nstep)
end = time.time()

# 動画をレンダリング
start_render = time.time()
framerate = 60
cam = mujoco.MjvCamera()
mujoco.mjv_defaultCamera(cam)
cam.distance = 0.2
cam.azimuth = 135
cam.elevation = -25
cam.lookat = [.2, -.2, 0.07]
models[0].vis.global_.fovy = 60
frames = render_many(models, top_data, state, framerate, camera=cam)
media.show_video(frames, fps=framerate)
end_render = time.time()

print(f'Rollout time {end-start:.1f} seconds')
print(f'Rendering time {end_render-start_render:.1f} seconds')

モデルが異なるようになったため、各ロールアウトの初期状態が同じであっても、ジャイロセンサーの測定値は一致しません。

In [0]:
plt.figure(figsize=(12, 8))
plt.subplot(3,1,1)
for i in range(nbatch): plt.plot(sensordata[i, :, 0])
plt.subplot(3,1,2)
for i in range(nbatch): plt.plot(sensordata[i, :, 1])
plt.subplot(3,1,3)
for i in range(nbatch): plt.plot(sensordata[i, :, 2])
plt.show()

### 例：制御入力
開ループ制御は `control` 引数を通じて `rollout` に渡すことができます。渡した場合、 `control` のサイズから推測できるため、 `nstep` を指定する必要はなくなります。

以下では、 [チュートリアルノートブック](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb) の手足をばたつかせるhumanoidを100体シミュレーションします。各humanoidは異なる制御信号を使用します。

In [0]:
# エピソードパラメータ。
duration = 3        # (seconds)
framerate = 120     # (Hz)

# 100個の異なる制御シーケンスを生成
nbatch = 100
nstep = int(duration / humanoid_model.opt.timestep)
times = np.linspace(0.0, duration, nstep)
ctrl_phase = 2 * np.pi * np.random.rand(nbatch, 1, humanoid_model.nu)
control = np.sin((2 * np.pi * times).reshape(nstep, 1) + ctrl_phase)

# 初期状態を作成
humanoid_data = mujoco.MjData(humanoid_model)
humanoid_data.qvel[2] = 4 # humanoidをジャンプさせる
initial_states = get_state(humanoid_model, humanoid_data, nbatch)
# インデックス0は時間なので、xとyのqpos値はインデックス1と2
initial_states[:, 1:3] = xy_grid(nbatch, ncols=10, spacing=1.0)


# ロールアウトを実行
start = time.time()
humanoid_datas = [copy.copy(humanoid_data) for _ in range(nthread)]
state, _ = rollout.rollout(humanoid_model, humanoid_datas,
                           initial_states, control)
end = time.time()

# ロールアウトをレンダリング
start_render = time.time()
framerate = 120
cam = mujoco.MjvCamera()
mujoco.mjv_defaultCamera(cam)
cam.distance = 10
cam.azimuth = 45
cam.elevation = -15
cam.lookat = [0, 0, 0]
humanoid_model.vis.global_.fovy = 60
frames = render_many(humanoid_model, humanoid_data, state, framerate,
                     camera=cam, light_pos=[0, 0, 10])
media.show_video(frames, fps=framerate/2) # 動画を半速で表示
end_render = time.time()

print(f'Rollout time {end-start:.1f} seconds')
print(f'Render time {end_render-start_render:.1f} seconds')

`rollout` の `control_spec` 引数を使用して、 `control` にアクチュエータ、一般化力、デカルト力、mocapポーズ、および/または等式制約の有効化/無効化の値が含まれていることを指定できます。内部的には [mj_setState](https://mujoco.readthedocs.io/en/stable/APIreference/APIfunctions.html#mj-setstate) を通じて管理され、 `control_spec` は `mj_setState` の `spec` 引数に対応します。

制御入力に加えてデカルト力を適用してみましょう。これにより、humanoidは手足を振りながら引きずられているように見えます。

In [0]:
xfrc_size = mujoco.mj_stateSize(humanoid_model, mujoco.mjtState.mjSTATE_XFRC_APPLIED)
xfrc = np.zeros((nbatch, nstep, xfrc_size))
head_id = humanoid_model.body('head').id

# 各モデルに一定だが異なる力を適用
force = np.random.normal(scale=150.0, size=(nbatch, 1, 3))
force[:,:,2] = 150  # 固定の垂直力
xfrc[:, :, 3*head_id:3*head_id+3] = force

control_xfrc = np.concatenate((control, xfrc), axis=2)
control_spec = mujoco.mjtState.mjSTATE_XFRC_APPLIED.value

start = time.time()
state, _ = rollout.rollout(humanoid_model, humanoid_datas,
                           initial_states, xfrc, control_spec=control_spec)
end = time.time()

start_render = time.time()
frames = render_many(humanoid_model, humanoid_data, state, framerate,
                     camera=cam, light_pos=[0, 0, 10])
media.show_video(frames, fps=framerate/2) # 動画を半速で表示
end_render = time.time()

print(f'Rollout time {end-start:.1f} seconds')
print(f'Render time {end_render-start_render:.1f} seconds')

# 高度な使い方

## skip_checks

デフォルトでは、rolloutは引数の次元に対して多くのチェックを行います。これにより、 `nbatch` や `nstep` などの次元を推測し、完全に指定されていない引数をタイル処理し、返される `state` と `sensordata` 配列を割り当てることができます。

しかし、これらのチェックには時間がかかります。特に `state` と `sensordata` が大きい場合や、多くのモデルがあり `nstep` が小さい場合に顕著です。そのため、上級ユーザーは追加のパフォーマンスを得るために `skip_checks=True` 引数を使用したい場合があります。

使用する場合、特定の引数が必須になり、すべてのシグナルが完全に定義されている必要があります（暗黙的なタイル処理なし）。具体的には：
* `model` は長さ `nbatch` のリストでなければなりません
* `data` は長さ `nthread` のリストでなければなりません
* `nstep` を指定する必要があります
* `initial_state` は `nbatch x nstate` の形状の配列でなければなりません
* `control` はオプションですが、渡す場合は `nbatch x nstep x ncontrol` の形状の配列でなければなりません
* `state` はオプションですが、状態を返す必要がある場合は渡す必要があり、 `nbatch x nstep x nstate` の形状でなければなりません
* `sensordata` はオプションですが、センサーデータを返す必要がある場合は渡す必要があり、 `nbatch x nstep x nsensordata` の形状でなければなりません

極端な例として、10,000個のhumanoidモデルを `rollout` に渡し、チェックあり/なしでそれぞれ1ステップずつシミュレーションします。

In [0]:
nbatch = 1000
nstep = [1, 10, 100, 500]
ntiming = 5

top_data = mujoco.MjData(top_model)
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
top_datas = [copy.copy(top_data) for _ in range(nthread)]
initial_state = get_state(top_model, top_data)
initial_state_tiled = get_state(top_model, top_data, nbatch)

# 注意: state, sensordata配列は自動的に割り当てられて返される
def rollout_with_checks(nstep):
  state, sensordata = rollout.rollout([top_model]*nbatch, top_datas, initial_state, nstep=nstep)

# 注意: state, sensordata配列は事前に割り当てる必要がある
state = None
sensordata = None
def rollout_skip_checks(nstep):
  # 注意: 初期状態はタイル処理する必要がある
  rollout.rollout([top_model]*nbatch, top_datas, initial_state_tiled, nstep=nstep,
                  state=state, sensordata=sensordata, skip_checks=True)

t_with_checks = benchmark(lambda x: rollout_with_checks(x), nstep, ntiming=ntiming)
t_skip_checks = benchmark(lambda x: rollout_skip_checks(x), nstep, ntiming=ntiming)

steps_per_second = (nbatch * np.array(nstep)) / np.array(t_with_checks)
steps_per_second_skip_checks = (nbatch * np.array(nstep)) / np.array(t_skip_checks)

plt.loglog(nstep, steps_per_second, label='with checks')
plt.loglog(nstep, steps_per_second_skip_checks, label='skip checks')
plt.ylabel('steps per second')
plt.xlabel('nstep')
ticker = matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ','))
plt.gca().yaxis.set_minor_formatter(ticker)
plt.legend()
plt.grid(True, which="both", axis="both")

予想通り、 `nstep` が増加するにつれて、skip checksを使用するメリットは急速に薄れます。しかし、nstepが小さくバッチサイズが大きい場合、大きな差が生じることがあります。

チェックありのバージョンではタイル処理されていない `initial_state` を使用できますが、skip checksバージョンではタイル処理された `initial_state_tiled` を使用する必要があることに注意してください。

## スレッドプールの再利用（ `Rollout` クラス）

`rollout` モジュールは `rollout` メソッドに加えて `Rollout` クラスを提供しています。 `Rollout` クラスは、内部で管理されるスレッドプールの安全な再利用を可能にするために設計されています。

再利用により、ロールアウトが短い場合に大幅な高速化が得られます。tippe topモデルでステップ数を増やしながら、高速化がどのように変化するか確認してみましょう。

In [0]:
nbatch = 100
nsteps = [2**i for i in [2, 3, 4, 5, 6, 7]]
ntiming = 5

top_data = mujoco.MjData(top_model)
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
top_datas = [copy.copy(top_data) for _ in range(nthread)]

initial_states = get_state(top_model, top_data, nbatch)

def rollout_method(nstep):
  for i in range(20):
    rollout.rollout(top_model, top_datas, initial_states, nstep=nstep)

def rollout_class(nstep):
  with rollout.Rollout(nthread=nthread) as rollout_:
    for i in range(20):
      rollout_.rollout(top_model, top_datas, initial_states, nstep=nstep)

t_method = benchmark(lambda x: rollout_method(x), nsteps, ntiming)
t_class = benchmark(lambda x: rollout_class(x), nsteps, ntiming)

plt.loglog(nsteps, nbatch * np.array(nsteps) / t_method, label='recreating threadpools')
plt.loglog(nsteps, nbatch * np.array(nsteps) / t_class, label='reusing threadpool')
plt.xlabel('nstep')
plt.ylabel('steps per second')
ticker = matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ','))
plt.gca().yaxis.set_minor_formatter(ticker)
plt.legend()
plt.grid(True, which="both", axis="both")

## スレッドプールの再利用（ `rollout` メソッド）

`rollout` は `persistent_pool=True` を渡すことで永続的なスレッドプールを作成して再利用します。ただし、いくつかの注意点があります。

まず、 `rollout` は関数であり、ユーザーがいつ呼び出しを終えるか分からないため、スレッドプールは以下のように手動でシャットダウンする必要があります：

In [0]:
nbatch = 1000
nstep = 1

top_data = mujoco.MjData(top_model)
mujoco.mj_resetDataKeyframe(top_model, top_data, 0)
top_datas = [copy.copy(top_data) for _ in range(nthread)]

initial_states = get_state(top_model, top_data, nbatch)

rollout.rollout(top_model, top_datas, initial_states, nstep=nstep, persistent_pool=True) # プールを作成
rollout.rollout(top_model, top_datas, initial_states, nstep=nstep, persistent_pool=True) # 以前作成したプールを再利用
rollout.shutdown_persistent_pool() # 完了したらプールを手動でシャットダウン

次に、 `rollout` が呼び出し間で同じスレッドプールを再利用する場合、複数のスレッドから `rollout` を呼び出すことは安全ではなくなります。例えば、以下は許可されていません（インタプリタのクラッシュを避けるため、問題のある行はコメントアウトされています）：

In [0]:
thread1 = threading.Thread(target=lambda: rollout.rollout(top_model, top_datas, initial_states, nstep=nstep, persistent_pool=True))
thread2 = threading.Thread(target=lambda: rollout.rollout(top_model, top_datas, initial_states, nstep=nstep, persistent_pool=True))

thread1.start()
#thread2.start() # これはしないでください！rolloutが2つのスレッドから同じ永続スレッドプールを使用し、インタプリタがクラッシュする可能性があります
thread1.join()
#thread2.join()
rollout.shutdown_persistent_pool()

## chunk_size

通信オーバーヘッドを最小化するため、 `rollout` はチャンクと呼ばれるロールアウトのグループ単位でスレッドにロールアウトを分配します。デフォルトでは、各チャンクに `max(1, 0.1 * (nbatch / nthread))` 個のロールアウトが割り当てられます。このチャンキングルールはほとんどのワークロードでうまく機能しますが、小さなモデルで短いロールアウトを行う場合など、常に最適とは限りません。

以下では、1000個のhopperを1ステップずつ実行する場合のchunk_sizeに対するステップ/秒をプロットします。この場合、デフォルトのchunk_sizeは、増加したチャンクサイズを使用するよりもかなり遅くなることが分かります。

In [0]:
nbatch = 100
nstep = 1
ntiming = 20

# モデルを読み込み
hopper_model = mujoco.MjModel.from_xml_path(hopper_path)
hopper_data = mujoco.MjData(hopper_model)
hopper_datas = [copy.copy(hopper_data) for _ in range(nthread)]

# 初期状態を取得
initial_states = get_state(hopper_model, hopper_data, nbatch)

def rollout_chunk_size(chunk_size=None):
  rollout.rollout(hopper_model, hopper_datas, initial_states, nstep=nstep, chunk_size=chunk_size)

# 異なるチャンクサイズでロールアウト
default_chunk_size = int(max(1.0, 0.1 * nbatch / nthread))
chunk_sizes = sorted([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, default_chunk_size])
t_chunk_size = benchmark(lambda x: rollout_chunk_size(x), chunk_sizes, ntiming=ntiming)

# 最適なチャンクサイズを取得
steps_per_second = nbatch * nstep / t_chunk_size
default_index = [i for i, c in enumerate(chunk_sizes) if c == default_chunk_size][0]
optimal_index = np.argmax(steps_per_second)
plt.loglog(chunk_sizes, steps_per_second, color='b')
plt.plot(chunk_sizes[default_index], steps_per_second[default_index], marker='o', color='r', label='default chunk size')
plt.plot(chunk_sizes[optimal_index], steps_per_second[optimal_index], marker='o', color='g', label='optimal chunk size')
plt.ylabel('steps per second')
plt.xlabel('chunk size')
ticker = matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ','))
plt.gca().yaxis.set_minor_formatter(ticker)
plt.legend()
plt.grid(True, which="both", axis="both")

print(f'default chunk size: {default_chunk_size} \t steps per second: {steps_per_second[default_index]:0.1f}')
print(f'optimal chunk size: {chunk_sizes[optimal_index]} \t steps per second: {steps_per_second[optimal_index]:0.1f}')

## ウォームスタート

`initial_warmstart` パラメータを使用して、ドキュメントの [計算の章](https://mujoco.readthedocs.io/en/stable/computation/index.html#warmstart-acceleration) で説明されているように制約ソルバーをウォームスタートできます。これは、モデルをステップのチャンクごとにロールアウトする場合に有用です。ウォームスタートがない場合、多体接触を含むカオス的なシステムは発散する可能性があります。

以下では、接触ソルバーをCGに変更したtippe topモデルでこれを実演します。これにより、デフォルトのニュートン法を使用する場合よりも接触力の計算の再現性が低くなり、ウォームスタートの利点を示すことができます。

シミュレーションは3回実行されます。1回目は6000ステップのロールアウト、2回目はウォームスタートありで60ステップを100チャンク、3回目はウォームスタートなしで60ステップを100チャンクです。

In [0]:
top_model_cg = copy.copy(top_model)

# CGソルバーに変更。ニュートンソルバーは収束が良すぎて
# ウォームスタートの効果が見えにくい
top_model_cg.opt.solver = mujoco.mjtSolver.mjSOL_CG

chunks = 100
steps_per_chunk = 60
nstep = steps_per_chunk*chunks

# 初期状態を取得
top_data_cg = mujoco.MjData(top_model_cg)
mujoco.mj_resetDataKeyframe(top_model_cg, top_data_cg, 0)
initial_state = get_state(top_model_cg, top_data_cg)

start = time.time()
# nstepステップでロールアウト
state_all, _  = rollout.rollout(top_model_cg, top_data_cg, initial_state, nstep=nstep)

# ウォームスタートありでチャンクごとにロールアウト
state_chunks = []
state_chunk, _ = rollout.rollout(top_model_cg, top_data_cg, initial_state, nstep=steps_per_chunk)
state_chunks.append(state_chunk)
for _ in range(chunks-1):
  state_chunk, _ = rollout.rollout(top_model_cg, top_data_cg, state_chunks[-1][0, -1, :],
                                   nstep=steps_per_chunk, initial_warmstart=top_data_cg.qacc_warmstart)
  state_chunks.append(state_chunk)
state_all_chunked_warmstart = np.concatenate(state_chunks, axis=1)

# ウォームスタートなしでチャンクごとにロールアウト
state_chunks = []
state_chunk, _ = rollout.rollout(top_model_cg, top_data_cg, initial_state, nstep=steps_per_chunk)
state_chunks.append(state_chunk)
first_warmstart = None
for i in range(chunks-1):
  state_chunk, _ = rollout.rollout(top_model_cg, top_data_cg, state_chunks[-1][0, -1, :], nstep=steps_per_chunk)
  state_chunks.append(state_chunk)
state_all_chunked = np.concatenate(state_chunks, axis=1)
end = time.time()

# ロールアウトをレンダリング
start_render = time.time()
framerate = 60
state_render = np.concatenate((state_all, state_all_chunked, state_all_chunked_warmstart), axis=0)
camera = 'distant'
frames1 = render_many(top_model_cg, top_data_cg, state_all, framerate, shape=(240, 320), transparent=False, camera=camera)
frames2 = render_many(top_model_cg, top_data_cg, state_all_chunked_warmstart, framerate, shape=(240, 320), transparent=False, camera=camera)
frames3 = render_many(top_model_cg, top_data_cg, state_all_chunked, framerate, shape=(240, 320), transparent=False, camera=camera)
media.show_video(np.concatenate((frames1, frames2, frames3), axis=2))
end_render = time.time()

print(f'Rollout took {end-start:.1f} seconds')
print(f'Rendering took {end_render-start_render:.1f} seconds')

予想通り、中央のアニメーション（ウォームスタートあり）は左の連続ロールアウトと一致しています。しかし、ウォームスタートを使用しなかったモデルは発散しました。

# ベンチマーク

`mujoco` Pythonライブラリの `rollout.rollout` 関数は、固定ステップ数のシミュレーションをバッチで実行します。シングルスレッドまたはマルチスレッドモードで実行できます。 `rollout` はマルチスレッドの使用を簡単に設定できるため、純粋なPythonに比べて大幅な高速化が得られます。

高速化を示すため、"tippe top"、"humanoid"、"humanoid100" モデルでベンチマークを実行します。

## Pythonロールアウト vs `rollout`

ベンチマークでは、3つのモデルに対してバッチ数とステップ数を変化させて実行します。

nbatch回のロールアウトをnstepステップ実行するPythonコードは以下の通りです：

In [0]:
def python_rollout(model, data, nbatch, nstep):
  for i in range(nbatch):
    for i in range(nstep):
      mujoco.mj_step(model, data)

`rollout` でnbatch回のロールアウトを実行するには、ロールアウトの開始点となるnbatch個の初期状態の配列を作成する必要があります。


さらに、 `rollout` の並列処理を使用するには、スレッドごとに1つの `MjData` を渡す必要があります。

`nbatch` 、 `nstep` 、 `nthread` でパラメータ化された `rollout` の呼び出しは以下の通りです：

In [0]:
def nthread_rollout(model, data, nbatch, nstep, nthread, rollout_):
  rollout_.rollout([model]*nbatch,
                   [copy.copy(data) for _ in range(nthread)], # Create one MjData per thread
                   np.tile(get_state(model, data), (nbatch, 1)), # Tile the initial condition nbatch times
                   nstep=nstep,
                   skip_checks=True)

次に、Pythonループと `rollout` をシングルスレッドおよびマルチスレッドモードの両方でベンチマークします。3つのベンチマークはAMD 5800X3Dで合計約2.5分かかります。

In [0]:
#@title ベンチマークユーティリティ

top_model = mujoco.MjModel.from_xml_string(tippe_top)
def init_top(model):
  data = mujoco.MjData(model)
  # 状態を回転するコマに設定（キーフレーム0）
  mujoco.mj_resetDataKeyframe(model, data, 0)
  return data

# humanoidモデルの作成と初期化
# ベンチマーク用の安定した接触セットを得るため2秒間ステップ
humanoid_model = mujoco.MjModel.from_xml_path(humanoid_path)
humanoid_data = mujoco.MjData(humanoid_model)
humanoid_data.qvel[2] = 4 # humanoidをジャンプさせる
while humanoid_data.time < 2.0:
  mujoco.mj_step(humanoid_model, humanoid_data)
humanoid_initial_state = get_state(humanoid_model, humanoid_data)
def init_humanoid(model):
  data = mujoco.MjData(model)
  mujoco.mj_setState(model, data, humanoid_initial_state.flatten(),
                     mujoco.mjtState.mjSTATE_FULLPHYSICS)
  return data

# humanoid100モデルの作成と初期化
# ベンチマーク用の安定した接触セットを得るため4秒間ステップ
humanoid100_model = mujoco.MjModel.from_xml_path(humanoid100_path)
humanoid100_data = mujoco.MjData(humanoid100_model)
while humanoid100_data.time < 4.0:
  mujoco.mj_step(humanoid100_model, humanoid100_data)
humanoid100_initial_state = get_state(humanoid100_model, humanoid100_data)
def init_humanoid100(model):
  data = mujoco.MjData(model)
  mujoco.mj_setState(model, data, humanoid100_initial_state.flatten(),
                     mujoco.mjtState.mjSTATE_FULLPHYSICS)
  return data

def benchmark_rollout(model, init_model, nbatch, nstep, nominal_nbatch, nominal_nstep, ntiming=1):
  print('Benchmarking pure python', end='\r')
  start = time.time()
  t_python_nbatch = benchmark(lambda x, data: python_rollout(model, data, x, nominal_nstep), nbatch, ntiming,
                              f_init=lambda x: init_model(model))
  t_python_nstep  = benchmark(lambda x, data: python_rollout(model, data, nominal_nbatch, x), nstep,  ntiming,
                              f_init=lambda x: init_model(model))
  end = time.time()
  print(f'Benchmarking pure python took {end-start:0.1f} seconds')

  print('Benchmarking single threaded rollout', end='\r')
  with rollout.Rollout(nthread=0) as rollout_:
    start = time.time()
    t_rollout_single_nbatch = benchmark(lambda x, data: nthread_rollout(model, data, x, nominal_nstep,  nthread=1, rollout_=rollout_),
                                        nbatch, ntiming,
                                        f_init=lambda x: init_model(model))
    t_rollout_single_nstep  = benchmark(lambda x, data: nthread_rollout(model, data, nominal_nbatch, x, nthread=1, rollout_=rollout_),
                                        nstep,  ntiming, f_init=lambda x: init_model(model))
    end = time.time()
  print(f'Benchmarking single threaded rollout took {end-start:0.1f} seconds')

  print(f'Benchmarking multithreaded rollout using {nthread} threads', end='\r')
  with rollout.Rollout(nthread=nthread) as rollout_:
    start = time.time()
    t_rollout_multi_nbatch = benchmark(lambda x, data: nthread_rollout(model, data, x, nominal_nstep,  nthread, rollout_=rollout_),
                                       nbatch, ntiming, f_init=lambda x: init_model(model))
    t_rollout_multi_nstep  = benchmark(lambda x, data: nthread_rollout(model, data, nominal_nbatch, x, nthread, rollout_=rollout_),
                                       nstep,  ntiming, f_init=lambda x: init_model(model))
    end = time.time()
  print(f'Benchmarking multithreaded rollout using {nthread} threads took {end-start:0.1f} seconds')

  return (t_python_nbatch, t_rollout_single_nbatch, t_rollout_multi_nbatch,
          t_python_nstep, t_rollout_single_nstep, t_rollout_multi_nstep)

def plot_benchmark(results, nbatch, nstep, nominal_nbatch, nominal_nstep, title):
  (t_python_nbatch, t_rollout_single_nbatch, t_rollout_multi_nbatch,
   t_python_nstep, t_rollout_single_nstep, t_rollout_multi_nstep) = results

  width = 0.25
  x = np.array([i for i in range(len(nbatch))])

  ticker = matplotlib.ticker.EngFormatter(unit='')

  fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
  steps_per_t = np.array(nbatch) * nominal_nstep
  steps_per_t_python = steps_per_t / t_python_nbatch
  steps_per_t_single = steps_per_t / t_rollout_single_nbatch
  steps_per_t_multi  = steps_per_t / t_rollout_multi_nbatch
  ax1.bar(x + 0*width, steps_per_t_python, width=width, label='python')
  ax1.bar(x + 1*width, steps_per_t_single, width=width, label='rollout single threaded')
  ax1.bar(x + 2*width, steps_per_t_multi, width=width, label='rollout multithreaded')
  ax1.set_xticks(x + width, nbatch)
  ax1.yaxis.set_major_formatter(ticker)
  ax1.grid()
  ax1.set_axisbelow(True)
  ax1.set_xlabel('nbatch')
  ax1.set_ylabel('steps per second')
  ax1.set_title(f'nbatch varied, nstep = {nominal_nstep}')

  x = np.array([i for i in range(len(nstep))])
  steps_per_t = np.array(nstep) * nominal_nbatch
  steps_per_t_python = steps_per_t / t_python_nstep
  steps_per_t_single = steps_per_t / t_rollout_single_nstep
  steps_per_t_multi  = steps_per_t / t_rollout_multi_nstep
  ax2.bar(x + 0*width, steps_per_t_python, width=width, label='python')
  ax2.bar(x + 1*width, steps_per_t_single, width=width, label='rollout single threaded')
  ax2.bar(x + 2*width, steps_per_t_multi, width=width, label='rollout multithreaded')
  ax2.set_xticks(x + width, nstep)
  ax2.yaxis.set_major_formatter(ticker)
  ax2.grid()
  ax2.set_axisbelow(True)
  ax2.set_xlabel('nstep')
  ax2.set_title(f'nstep varied, nbatch = {nominal_nbatch}')

  ax1.legend(loc=(0.03, 0.8))
  fig.set_size_inches(10, 5)
  plt.suptitle(title)
  plt.tight_layout()

### Tippe Topベンチマーク

In [0]:
nominal_nbatch = 256 # 異なるnstepをテストする際に使用するバッチサイズ
nominal_nstep = 5 # 異なるnbatchをテストする際に使用するステップ数
nbatch = [1, 256, 2048, 8192]
nstep = [1, 10, 100, 1000]

top_benchmark_results = benchmark_rollout(top_model, init_top,
                                          nbatch, nstep,
                                          nominal_nbatch, nominal_nstep)
plot_benchmark(top_benchmark_results, nbatch, nstep,
               nominal_nbatch, nominal_nstep,
               title='Tippe Top')

### Humanoidベンチマーク

In [0]:
nominal_nbatch = 256 # 異なるnstepをテストする際に使用するバッチサイズ
nominal_nstep = 5 # 異なるnbatchをテストする際に使用するステップ数
nbatch = [1, 256, 2048, 8192] # ベンチマークするバッチサイズ
nstep = [1, 10, 100, 1000] # ベンチマークするステップ数

humanoid_benchmark_results = benchmark_rollout(humanoid_model, init_humanoid,
                                               nbatch, nstep,
                                               nominal_nbatch, nominal_nstep)
plot_benchmark(humanoid_benchmark_results, nbatch, nstep,
               nominal_nbatch, nominal_nstep,
               title='Humanoid')

### Humanoid100ベンチマーク

In [0]:
nominal_nbatch = 128 # 異なるnstepをテストする際に使用するバッチサイズ
nominal_nstep = 5 # 異なるnbatchをテストする際に使用するステップ数
nbatch = [1, 64, 128, 256] # ベンチマークするバッチサイズ
nstep = [1, 10, 100, 1000] # ベンチマークするステップ数

humanoid100_benchmark_results = benchmark_rollout(
    humanoid100_model,
    init_humanoid100,
    nbatch,
    nstep,
    nominal_nbatch,
    nominal_nstep,
)
plot_benchmark(humanoid100_benchmark_results, nbatch, nstep,
               nominal_nbatch, nominal_nstep,
               title='Humanoid100')

# MJX vs `rollout`

次に、tippe topとhumanoidモデルを使用して `rollout` とMJXをベンチマーク比較します（humanoid100はMJXではサポートされていません）。

次の2つのベンチマークは、AMD 5800X3DとNVIDIA 4090で合計約16.5分かかります。ほとんどの時間はMJX関数のJITコンパイルに費やされます。JIT関数はキャッシュされるため、ベンチマークの後続の実行はより高速になります。

**注意：** MJXは、ニューラルネットワークのようなGPUで最も効果を発揮する他のものと組み合わせて使用する場合に最も有用です。そのような追加のワークロードがない場合、CPUベースのシミュレーションの方が高速になることがあります。特に、最新のGPUを使用していない場合に顕著です。

In [0]:
#@title MJXヘルパー関数
def init_mjx_batch(model, init_model, nbatch, nstep, skip_jit=False):
  data = init_model(model)

  # モデルとデータのMJXバージョンを作成
  mjx_model = mjx.put_model(model)
  mjx_data = mjx.put_data(model, data)

  batch = jax.vmap(lambda x: mjx_data)(jp.array(list(range(nbatch))))
  jax.block_until_ready(batch)

  if not skip_jit:
    start = time.time()
    jit_step = jax.vmap(mjx.step, in_axes=(None, 0))
    def unroll(d, _):
      d = jit_step(mjx_model, d)
      return d, None
    jit_unroll = jax.jit(lambda d: jax.lax.scan(unroll, d, None, length=nstep, unroll=4)[0])
    jit_unroll = jit_unroll.lower(batch).compile()
    end = time.time()
    jit_time = end - start
  else:
    jit_unroll = None
    jit_time = 0.0

  return mjx_model, mjx_data, jit_unroll, batch, jit_time

def mjx_rollout(batch, jit_unroll):
  batch = jit_unroll(batch)
  jax.block_until_ready(batch)

def benchmark_mjx(model, init_model, nbatch, nstep, nominal_nbatch, nominal_nstep, ntiming=1, jit_unroll_cache=None):
  print(f'Benchmarking multithreaded rollout using {nthread} threads', end="\r")
  with rollout.Rollout(nthread=nthread) as rollout_:
    start = time.time()
    t_rollout_multi_nbatch = benchmark(lambda x, data: nthread_rollout(model, data, x, nominal_nstep,  nthread, rollout_),
                                       nbatch, ntiming, f_init=lambda x: init_model(model))
    t_rollout_multi_nstep  = benchmark(lambda x, data: nthread_rollout(model, data, nominal_nbatch, x, nthread, rollout_),
                                       nstep,  ntiming, f_init=lambda x: init_model(model))
    end = time.time()
  print(f'Benchmarking multithreaded rollout using {nthread} threads took {end-start:0.1f} seconds')

  print('Running JIT for MJX', end='\r')
  total_jit = 0.0
  if jit_unroll_cache is None:
    jit_unroll_cache = {}
  if f'nbatch_{nominal_nstep}' not in jit_unroll_cache:
    jit_unroll_cache[f'nbatch_{nominal_nstep}'] = {}
  if f'nstep_{nominal_nbatch}' not in jit_unroll_cache:
    jit_unroll_cache[f'nstep_{nominal_nbatch}'] = {}
  for n in nbatch:
    if n not in jit_unroll_cache[f'nbatch_{nominal_nstep}']:
      _, _, jit_unroll_cache[f'nbatch_{nominal_nstep}'][n], _, jit_time = init_mjx_batch(model, init_model, n, nominal_nstep)
      total_jit += jit_time
  for n in nstep:
    if n not in jit_unroll_cache[f'nstep_{nominal_nbatch}']:
      _, _, jit_unroll_cache[f'nstep_{nominal_nbatch}'][n], _, jit_time = init_mjx_batch(model, init_model, nominal_nbatch, n)
      total_jit += jit_time
  print(f'Running JIT for MJX took {total_jit:0.1f} seconds')

  print('Benchmarking MJX', end='\r')
  start = time.time()
  t_mjx_nbatch = benchmark(lambda x, x_init: mjx_rollout(x_init[3], jit_unroll_cache[f'nbatch_{nominal_nstep}'][x]),
                           nbatch, ntiming, f_init=lambda x: init_mjx_batch(model, init_model, x, nominal_nstep, skip_jit=True))
  t_mjx_nstep  = benchmark(lambda x, x_init: mjx_rollout(x_init[3], jit_unroll_cache[f'nstep_{nominal_nbatch}'][x]),
                           nstep, ntiming, f_init=lambda x: init_mjx_batch(model, init_model, nominal_nbatch, x, skip_jit=True))
  end = time.time()
  print(f'Benchmarking MJX took {end-start:0.1f} seconds')

  return t_rollout_multi_nbatch, t_rollout_multi_nstep, t_mjx_nbatch, t_mjx_nstep

def plot_mjx_benchmark(results, nbatch, nstep, nominal_nbatch, nominal_nstep, title):
  t_rollout_multi_nbatch, t_rollout_multi_nstep, t_mjx_nbatch, t_mjx_nstep = results

  width = 0.333
  x = np.array([i for i in range(len(nbatch))])

  ticker = matplotlib.ticker.EngFormatter(unit='')

  fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
  steps_per_t = np.array(nbatch) * nominal_nstep
  steps_per_t_mjx = steps_per_t / t_mjx_nbatch
  steps_per_t_multi  = steps_per_t / t_rollout_multi_nbatch
  ax1.bar(x + 0*width, steps_per_t_mjx, width=width, label='mjx')
  ax1.bar(x + 1*width, steps_per_t_multi, width=width, label='rollout multithreaded')
  ax1.set_xticks(x + width / 2, nbatch)
  ax1.yaxis.set_major_formatter(ticker)
  ax1.grid()
  ax1.set_xlabel('nbatch')
  ax1.set_ylabel('steps per second')
  ax1.set_title(f'nbatch varied, nstep = {nominal_nstep}')

  x = np.array([i for i in range(len(nstep))])
  steps_per_t = np.array(nstep) * nominal_nbatch
  steps_per_t_mjx = steps_per_t / t_mjx_nstep
  steps_per_t_multi  = steps_per_t / t_rollout_multi_nstep
  ax2.bar(x + 0*width, steps_per_t_mjx, width=width, label='mjx')
  ax2.bar(x + 1*width, steps_per_t_multi, width=width, label='rollout multithreaded')
  ax2.set_xticks(x + width / 2, nstep)
  ax2.yaxis.set_major_formatter(ticker)
  ax2.grid()
  ax2.set_xlabel('nstep')
  ax2.set_title(f'nstep varied, nbatch = {nominal_nbatch}')

  ax2.legend(loc=(1.04, 0.0))
  fig.set_size_inches(10, 4)
  plt.suptitle(title)
  plt.tight_layout()

# jit_step関数のキャッシュ。コンパイルに長時間かかる
top_jit_unroll_cache = {}
humanoid_jit_unroll_cache = {}

### MJX Tippe Topベンチマーク

In [0]:
nominal_nbatch = 16384 # 異なるnstepをテストする際に使用するバッチサイズ
nominal_nstep = 5 # 異なるnbatchをテストする際に使用するステップ数
nbatch = [4096, 16384, 65536, 131072] # ベンチマークするバッチサイズ
nstep = [1, 10, 100, 200] # ベンチマークするステップ数

mjx_top_results = benchmark_mjx(top_model, init_top, nbatch, nstep, nominal_nbatch, nominal_nstep,
                                jit_unroll_cache=top_jit_unroll_cache)
plot_mjx_benchmark(mjx_top_results, nbatch, nstep, nominal_nbatch, nominal_nstep, title='MJX Tippe Top')

### MJX Humanoidベンチマーク

In [0]:
nominal_nbatch = 4096 # 異なるnstepをテストする際に使用するバッチサイズ
nominal_nstep = 5 # 異なるnbatchをテストする際に使用するステップ数
nbatch = [1024, 4096, 16384, 32768] # ベンチマークするバッチサイズ
nstep = [1, 10, 100, 200] # ベンチマークするステップ数

mjx_humanoid_results = benchmark_mjx(humanoid_model, init_humanoid, nbatch, nstep, nominal_nbatch, nominal_nstep,
                                     jit_unroll_cache=humanoid_jit_unroll_cache)
plot_mjx_benchmark(mjx_humanoid_results, nbatch, nstep, nominal_nbatch, nominal_nstep, title='MJX Humanoid')

### MJX 1つのモデル内の複数Humanoid

MJXの [ドキュメント](https://mujoco.readthedocs.io/en/stable/mjx.html#mjx-the-sharp-bits) には、様々なデバイスでのネイティブMuJoCo vs MJXの速度比較チャートが掲載されています。

ここでは、MJXと `rollout` を比較するための同様のプロットを作成します。5800X3Dと4090では、ベンチマークの実行に約16.5分かかります。

**注意：** これらの結果は、ドキュメントのプロットとは直接比較できません。特に、バッチサイズが4090に収めるために8192から4096に削減されています。

In [0]:
max_humanoids = 10
nbatch = 8192 // 2 # 元のベンチマークはバッチサイズ8192で実行されたが、4090では約4096個のhumanoidしか収まらない
nstep = 200

jit_step = jax.vmap(mjx.step, in_axes=(None, 0))
t_rollout = []
t_mjx = []
for i in range(1, max_humanoids+1):
  print(f'Running benchmark on {i} humanoids')
  nhumanoid_model = mujoco.MjModel.from_xml_path(
      f'mujoco/mjx/mujoco/mjx/test_data/humanoid/{i:02d}_humanoids.xml'
  )
  nhumanoid_data = mujoco.MjData(nhumanoid_model)

  mjx_model = mjx.put_model(nhumanoid_model)
  mjx_data = mjx.put_data(nhumanoid_model, nhumanoid_data)
  batch = jax.vmap(lambda x: mjx_data)(jp.array(list(range(nbatch))))
  jax.block_until_ready(batch)

  with rollout.Rollout(nthread=nthread) as rollout_:
    initial_state = get_state(nhumanoid_model, nhumanoid_data, nbatch)
    start = time.perf_counter()
    rollout_.rollout([nhumanoid_model]*nbatch,
                     [copy.copy(nhumanoid_data) for _ in range(nthread)],
                     initial_state=initial_state,
                     nstep=nstep, skip_checks=True)
    end = time.perf_counter()
  t_rollout.append(end-start)

  # ベンチマーク情報にJIT時間を含めないよう、モデル/バッチのJITをトリガー
  def unroll(d, _):
    d = jit_step(mjx_model, d)
    return d, None
  jit_unroll = jax.jit(lambda d: jax.lax.scan(unroll, d, None, length=nstep, unroll=4)[0])
  jit_unroll = jit_unroll.lower(batch).compile()

  start = time.perf_counter()
  jit_unroll(batch)
  jax.block_until_ready(batch)
  end = time.perf_counter()
  t_mjx.append(end-start)

In [0]:
#@title MJX nhumanoidベンチマークのプロット

def plot_mjx_nhumanoid_benchmark(t_rollout, t_mjx, nbatch, nstep, max_humanoids):
  nhumanoids = [i for i in range(1, max_humanoids+1)]

  width = 0.333
  x = np.array([i for i in range(len(nhumanoids))])

  ticker = matplotlib.ticker.EngFormatter(unit='')

  fig, ax1 = plt.subplots(1, 1, sharey=True)
  steps_per_t = nbatch * nstep
  steps_per_t_mjx = steps_per_t / np.array(t_mjx)
  steps_per_t_multi  = steps_per_t / np.array(t_rollout)
  ax1.bar(x + 0*width, steps_per_t_mjx, width=width, label='mjx')
  ax1.bar(x + 1*width, steps_per_t_multi, width=width, label='rollout multithreaded')
  ax1.set_xticks(x + width / 2, nhumanoids)
  ax1.yaxis.set_major_formatter(ticker)
  ax1.set_yscale('log')
  ax1.grid()
  ax1.set_xlabel('number of humanoids')
  ax1.set_ylabel('steps per second')
  ax1.set_title(f'nhumanoids varied, nbatch = {nbatch}, nstep = {nstep}')

  ax1.legend(loc=(1.04, 0.0))
  fig.set_size_inches(8, 4)
  plt.tight_layout()

plot_mjx_nhumanoid_benchmark(t_rollout, t_mjx, nbatch, nstep, max_humanoids)